In [1]:
import sys
import pickle
import json
import pandas as pd
import numpy as np
sys.path.append('../')
from sklearn.pipeline import Pipeline
from modules.transformers.CenterPosition import CenterPositionTransform
from modules.transformers.TextType import isdateTransform, isfloatTransform,istextTransform
from modules.transformers.textalignment import AlignmentTransform
pd.set_option('display.max_rows', 1000)
# from trcode.line import Line
# from trcode import helper
# from trcode.info_word import InfoWord

In [2]:
SYMBOL = {
    "$": 0,
}
sfea_words = SYMBOL

def clean_word(word):
    """
    Clean up a word
    :param word: input wordText
    :return: cleaned up word string
    """
    cword = word.strip(' /)*%-_:;\'\"')
    cword = cword.replace(',', '')
    cword = cword.replace('.', '')
    cword = cword.replace('(', '')
    cword = cword.replace('$', '')
    return cword
def gen_raw_feature(words):
    """
    Input a word list from one page, output preliminary features for this page.
    :param words: A list of strings which are words from this page.
    :return: A list of preliminary features contains word frequencies and other features like word count.
                Need some cleanup afterwards.
    """
    word_count = 0
    number_count = 0
    fea_sym = [0]*(max(sfea_words.values())+1)
    for w in words:
        # Count '$'
        dn = w.count('$')
        fea_sym[sfea_words['$']] += dn
        if dn > 1:
            wl = w.split('$')
            w = wl[0]
            words += wl[1:]
        cw = clean_word(w)
        if not cw:
            continue
        # Increase word count
        word_count += 1
        # Check if numbers and increase number count
        if cw.isnumeric():
            number_count += 1
            continue
    # Construct other features
    if not word_count:
        fea_oth = [0, 0, 0]
    else:
        fea_oth = [word_count, number_count]
    return fea_sym + fea_oth

In [3]:
def read_pages(ticker):
    with open('C:\\Users\\juang\\Documents\\Stocks\\PDFs\\'+ticker+'\\all.json', 'r') as f:
        datastore = json.load(f)
    df = pd.DataFrame()
    for i in range(0, len(datastore['Pages'])):
        df = df.append(pd.DataFrame(datastore['Pages'][i]))
    df = df[df.index == 0].set_index('PageNumber')
    df.index = df.index.astype(int)
    df = df.sort_index()
    df['Dollar_Count'] = ''
    df['Word_Count'] = ''
    df['Number_Count'] = ''
    for i in range(0, len(df)):
        features = gen_raw_feature(df.WordText[i])
        df['Dollar_Count'][i] = features[0]
        df['Word_Count'][i] = features[1]
        df['Number_Count'][i] = features[2]
    return df

In [4]:
def read_words(ticker):
    with open('C:\\Users\\juang\\Documents\\Stocks\\PDFs\\'+ticker+'\\all.json', 'r') as f:
        datastore = json.load(f)
    df = pd.DataFrame()
    for i in range(0, len(datastore['Pages'])):
        df = df.append(pd.DataFrame(datastore['Pages'][i]))
    df = df[df.index != 0].set_index('PageNumber')
    df.index = df.index.astype(int)
    df = df.sort_index()
    return df

In [5]:
def predict_pages(df):
    array = [df.Number_Count/df.Word_Count, df.Dollar_Count]
    g = number_clf.predict_proba(np.transpose(array))
    m = text_clf.predict_proba(df.WordText)
    all_probas = np.hstack((m, g))
    all_probas
    return final_clf.predict(all_probas)

In [6]:
with open('text_clf.pickle', 'rb') as f:
    text_clf = pickle.load(f)
with open('number_clf.pickle', 'rb') as f:
    number_clf = pickle.load(f)
with open('final_clf.pickle', 'rb') as f:
    final_clf = pickle.load(f)

In [7]:
chk = read_pages('CHK')

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [55]:
lister = predict_pages(chk)
index = []
financials = [index.append(i) for i in range(0,len(lister)) if lister[i]==1]
index

[2, 3, 4, 6, 8, 10]

In [78]:
chk_words = read_words('CHK').reset_index()
chk_words = chk_words[chk_words.PageNumber.isin(index)]
chk_words

,PageNumber,WordText,X_Max,X_Min,Y_Max,Y_Min
529,2,at,744.0,717.0,1345.0,1320.0
530,2,"depreciation,",655.0,472.0,1400.0,1368.0
531,2,Cost,823.0,755.0,1345.0,1318.0
532,2,Less:,265.0,190.0,1395.0,1368.0
533,2,"Equipment,",704.0,542.0,1352.0,1318.0
534,2,accumulated,461.0,277.0,1395.0,1368.0
535,2,depletion,797.0,666.0,1402.0,1368.0
536,2,and,376.0,325.0,1441.0,1406.0
537,2,amortization,1047.0,868.0,1395.0,1368.0
538,2,"(62,726)",2329.0,2211.0,1449.0,1403.0


In [80]:
chk_words = chk_words.reset_index()
chk_words['PageNumber'] = chk_words['PageNumber'].astype(int)
chk_words['index'] = chk_words['index'].astype(int)
chk_words['index'] = 1
chk_words

,index,PageNumber,WordText,X_Max,X_Min,Y_Max,Y_Min
0,1,2,at,744.0,717.0,1345.0,1320.0
1,1,2,"depreciation,",655.0,472.0,1400.0,1368.0
2,1,2,Cost,823.0,755.0,1345.0,1318.0
3,1,2,Less:,265.0,190.0,1395.0,1368.0
4,1,2,"Equipment,",704.0,542.0,1352.0,1318.0
5,1,2,accumulated,461.0,277.0,1395.0,1368.0
6,1,2,depletion,797.0,666.0,1402.0,1368.0
7,1,2,and,376.0,325.0,1441.0,1406.0
8,1,2,amortization,1047.0,868.0,1395.0,1368.0
9,1,2,"(62,726)",2329.0,2211.0,1449.0,1403.0


In [105]:
intakePipeSteps = []
intakePipeSteps.append(('line',AlignmentTransform(['Y_Max','Y_Min','index','PageNumber'],output_name='line',sort_columns=['index','PageNumber','Y_Min'],sort_values=[True,True,True])))
intakePipeSteps.append(('CenterTransform',CenterPositionTransform()))
intakePipeSteps.append(('isfloat',isfloatTransform('WordText')))
intakePipeSteps.append(('isdate',isdateTransform('WordText')))
intakePipeSteps.append(('istext',istextTransform('WordText')))
combined = Pipeline(intakePipeSteps)
combined

Pipeline(steps=[('line', <modules.transformers.textalignment.AlignmentTransform object at 0x000001B902BF5438>), ('CenterTransform', <modules.transformers.CenterPosition.CenterPositionTransform object at 0x000001B902BF57B8>), ('isfloat', <modules.transformers.TextType.isfloatTransform object at 0x000001B902BF59B0>), ('isdate', <modules.transformers.TextType.isdateTransform object at 0x000001B902DC4A20>), ('istext', <modules.transformers.TextType.istextTransform object at 0x000001B902BF5710>)])

In [106]:
date = combined.named_steps['isdate'].transform(chk_words)
floater = combined.named_steps['isfloat'].transform(chk_words)
text = combined.named_steps['istext'].transform(chk_words)
center = combined.named_steps['CenterTransform'].transform(chk_words)

1354 13 TextAlignment in
1354 13 isdate in
1354 13 isfloat in
1354 13 istext in
1354 13 centerposition in


In [102]:
# chk_words['line'] = list(line.line)
chk_words = chk_words.sort_values(['PageNumber','Y_Min','X_Min'])

In [104]:
chk_words.groupby(['PageNumber','Y_Min']).apply(lambda x: x.sum())

index  PageNumber  \
PageNumber Y_Min                       
2          118.0     1.0         2.0   
           149.0     4.0         8.0   
           330.0     5.0        10.0   
           399.0     5.0        10.0   
           436.0     4.0         8.0   
           471.0     1.0         2.0   
           578.0     4.0         8.0   
           616.0     1.0         2.0   
           617.0     1.0         2.0   
           669.0     2.0         4.0   
           671.0     1.0         2.0   
           721.0     2.0         4.0   
           769.0     2.0         4.0   
           770.0     8.0        16.0   
           772.0     1.0         2.0   
           775.0     1.0         2.0   
           815.0     1.0         2.0   
           817.0     1.0         2.0   
           822.0     2.0         4.0   
           823.0     1.0         2.0   
           870.0     1.0         2.0   
           871.0     3.0         6.0   
           918.0     3.0         6.0   
           921.0     2.0         4.0   
           971.0     1.0         2.0   
           972.0     4.0         8.0   
           1027.0    3.0         6.0   
           1075.0    9.0        18.0   
           1077.0    1.0         2.0   
           1082.0    2.0         4.0   
           1125.0    4.0         8.0   
           1127.0    1.0         2.0   
           1132.0    1.0         2.0   
           1165.0    2.0         4.0   
           1166.0    5.0        10.0   
           1168.0    1.0         2.0   
           1173.0    1.0         2.0   
           1210.0    1.0         2.0   
           1216.0    3.0         6.0   
           1264.0    1.0         2.0   
           1265.0    4.0         8.0   
           1267.0    1.0         2.0   
           1315.0    1.0         2.0   
           1317.0    1.0         2.0   
           1318.0    5.0        10.0   
           1320.0    1.0         2.0   
           1368.0    6.0        12.0   
           1403.0    1.0         2.0   
           1406.0    7.0        14.0   
           1407.0    1.0         2.0   
           1457.0    1.0         2.0   
           1459.0    1.0         2.0   
           1464.0    7.0        14.0   
           1509.0    1.0         2.0   
           1510.0    3.0         6.0   
           1512.0    3.0         6.0   
           1563.0    2.0         4.0   
           1611.0    1.0         2.0   
           1613.0    3.0         6.0   
           1615.0    1.0         2.0   
           1664.0    2.0         4.0   
           1666.0    1.0         2.0   
           1668.0    2.0         4.0   
           1810.0    9.0        18.0   
           1812.0    2.0         4.0   
           1817.0    2.0         4.0   
           1848.0    1.0         2.0   
3          118.0     1.0         3.0   
           149.0     5.0        15.0   
           184.0     6.0        18.0   
           224.0     1.0         3.0   
           330.0     4.0        12.0   
           367.0     1.0         3.0   
           368.0     1.0         3.0   
           421.0     1.0         3.0   
           423.0     2.0         6.0   
           474.0     2.0         6.0   
           519.0     1.0         3.0   
           520.0     1.0         3.0   
           522.0     2.0         6.0   
           524.0     1.0         3.0   
           573.0     6.0        18.0   
           575.0     1.0         3.0   
           621.0     2.0         6.0   
           623.0     2.0         6.0   
           673.0     4.0        12.0   
           719.0     1.0         3.0   
           721.0    11.0        33.0   
           773.0     1.0         3.0   
           774.0     1.0         3.0   
           775.0     3.0         9.0   
           829.0     2.0         6.0   
           875.0     1.0         3.0   
           878.0     3.0         9.0   
           880.0     1.0         3.0   
           928.0     4.0        12.0   
           974.0     1.0         3.0   
           976.0     1.0         3.0   
           978.0   

In [195]:
def text_align(df):
    newlist = []
    previous = 0
    count = 0
    for row in df.Y_Min:
        if count == 0:
            previous = row
            newlist.append(previous)
        else:
            if abs(row - previous) > 10:
                previous = row
                newlist.append(row)
            else:
                row = previous
                newlist.append(row)
        count = count + 1
    return newlist

In [197]:
chk_words['Y_Min'] = text_align(chk_words)

In [198]:
chk_words.groupby(['PageNumber','Y_Min']).apply(lambda x: x.sum())

index  PageNumber  \
PageNumber Y_Min                       
2          118.0     1.0         2.0   
           149.0     4.0         8.0   
           330.0     5.0        10.0   
           399.0     5.0        10.0   
           436.0     4.0         8.0   
           471.0     1.0         2.0   
           578.0     4.0         8.0   
           616.0     2.0         4.0   
           669.0     3.0         6.0   
           721.0     2.0         4.0   
           769.0    12.0        24.0   
           815.0     5.0        10.0   
           870.0     4.0         8.0   
           918.0     5.0        10.0   
           971.0     5.0        10.0   
           1027.0    3.0         6.0   
           1075.0   12.0        24.0   
           1125.0    6.0        12.0   
           1165.0    9.0        18.0   
           1210.0    4.0         8.0   
           1264.0    6.0        12.0   
           1315.0    8.0        16.0   
           1368.0    6.0        12.0   
           1403.0    9.0        18.0   
           1457.0    9.0        18.0   
           1509.0    7.0        14.0   
           1563.0    2.0         4.0   
           1611.0    5.0        10.0   
           1664.0    5.0        10.0   
           1810.0   13.0        26.0   
           1848.0    1.0         2.0   
3          118.0     1.0         3.0   
           149.0     5.0        15.0   
           184.0     6.0        18.0   
           224.0     1.0         3.0   
           330.0     4.0        12.0   
           367.0     2.0         6.0   
           421.0     3.0         9.0   
           474.0     2.0         6.0   
           519.0     5.0        15.0   
           573.0     7.0        21.0   
           621.0     4.0        12.0   
           673.0     4.0        12.0   
           719.0    12.0        36.0   
           773.0     5.0        15.0   
           829.0     2.0         6.0   
           875.0     5.0        15.0   
           928.0     4.0        12.0   
           974.0     9.0        27.0   
           1027.0    4.0        12.0   
           1077.0    5.0        15.0   
           1133.0    5.0        15.0   
           1183.0    1.0         3.0   
           1233.0    3.0         9.0   
           1281.0    8.0        24.0   
           1318.0    7.0        21.0   
           1372.0    5.0        15.0   
           1413.0    5.0        15.0   
           1454.0    5.0        15.0   
           1500.0    5.0        15.0   
           1552.0    4.0        12.0   
           1601.0    6.0        18.0   
           1652.0    5.0        15.0   
           1690.0    6.0        18.0   
           1744.0    7.0        21.0   
           1798.0    4.0        12.0   
           1851.0    5.0        15.0   
           1901.0    7.0        21.0   
           1995.0   13.0        39.0   
           2033.0    1.0         3.0   
4          118.0     1.0         4.0   
           149.0     5.0        20.0   
           184.0     5.0        20.0   
           224.0     1.0         4.0   
           368.0     6.0        24.0   
           409.0     4.0        16.0   
           459.0     4.0        16.0   
           512.0     7.0        28.0   
           562.0     1.0         4.0   
           610.0     7.0        28.0   
           659.0     7.0        28.0   
           712.0     6.0        24.0   
           769.0     2.0         8.0   
           818.0     8.0        32.0   
           865.0    12.0        48.0   
           916.0     4.0        16.0   
           964.0     8.0        32.0   
           1019.0    5.0        20.0   
           1068.0    5.0        20.0   
           1118.0    6.0        24.0   
           1168.0    9.0        36.0   
           1217.0    8.0        32.0   
           1267.0    8.0        32.0   
           1318.0    7.0        28.0   
           1368.0    7.0        28.0   
           1418.0    7.0        28.0   
           1470.0    7.0        28.0   
           1526.0    3.0        12.0   
           1575.0  